<a href="https://colab.research.google.com/github/tchtan/Taxi-Analytics/blob/main/taxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Lib**

In [ ]:
import pandas as pd
import numpy as np
import shapely.geometry as geometry
from tqdm import tqdm
from datetime import datetime
from math import radians, cos, sin, asin, sqrt

# **Haversine Formula**

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    km = 6371* c
    m = km * 1000
    return m

# **BKK Polygon**

In [ ]:
polygoncoords = [
    (100.3260000, 13.8080000),
    (100.3240000, 13.8040000),
    (100.3290000, 13.7860000),
    (100.3260000, 13.7710000),
    (100.3260000, 13.7610000),
    (100.3290000, 13.7580000),
    (100.3290000, 13.7530000),
    (100.3320000, 13.7510000),
    (100.3310000, 13.7450000),
    (100.3330000, 13.7360000),
    (100.3310000, 13.7340000),
    (100.3320000, 13.7250000),
    (100.3290000, 13.7230000),
    (100.3310000, 13.7160000),
    (100.3280000, 13.7140000),
    (100.3290000, 13.7070000),
    (100.3270000, 13.6950000),
    (100.3300000, 13.6790000),
    (100.3340000, 13.6740000),
    (100.3370000, 13.6630000),
    (100.3350000, 13.6540000),
    (100.3380000, 13.6510000),
    (100.3420000, 13.6510000),
    (100.3420000, 13.6420000),
    (100.3470000, 13.6370000),
    (100.3510000, 13.6360000),
    (100.3530000, 13.6330000),
    (100.3560000, 13.6330000),
    (100.3580000, 13.6290000),
    (100.3630000, 13.6290000),
    (100.3640000, 13.6270000),
    (100.3660000, 13.6270000),
    (100.3680000, 13.6190000),
    (100.3700000, 13.6170000),
    (100.3730000, 13.6170000),
    (100.3750000, 13.5920000),
    (100.3810000, 13.5860000),
    (100.3810000, 13.5780000),
    (100.3870000, 13.5670000),
    (100.3870000, 13.5630000),
    (100.3920000, 13.5510000),
    (100.3960000, 13.5490000),
    (100.4040000, 13.5490000),
    (100.4110000, 13.5510000),
    (100.4110000, 13.5490000),
    (100.4160000, 13.5450000),
    (100.4090000, 13.5380000),
    (100.4070000, 13.5260000),
    (100.4030000, 13.5210000),
    (100.4030000, 13.5110000),
    (100.4010000, 13.5100000),
    (100.4010000, 13.5070000),
    (100.4540000, 13.2160000),
    (100.4560000, 13.2130000),
    (100.4610000, 13.2130000),
    (100.4620000, 13.2170000),
    (100.4580000, 13.4980000),
    (100.4480000, 13.5540000),
    (100.4510000, 13.5820000),
    (100.4500000, 13.5850000),
    (100.4550000, 13.5930000),
    (100.4560000, 13.6010000),
    (100.4580000, 13.6000000),
    (100.4620000, 13.6010000),
    (100.4630000, 13.5950000),
    (100.4660000, 13.5920000),
    (100.4660000, 13.5900000),
    (100.4710000, 13.5890000),
    (100.4770000, 13.5910000),
    (100.4780000, 13.5950000),
    (100.4770000, 13.5970000),
    (100.4830000, 13.5970000),
    (100.4870000, 13.5990000),
    (100.4890000, 13.6020000),
    (100.4980000, 13.6020000),
    (100.5010000, 13.6040000),
    (100.5040000, 13.6020000),
    (100.5050000, 13.5960000),
    (100.5080000, 13.5930000),
    (100.5140000, 13.5930000),
    (100.5170000, 13.6000000),
    (100.5240000, 13.6020000),
    (100.5250000, 13.6070000),
    (100.5240000, 13.6110000),
    (100.5210000, 13.6140000),
    (100.5190000, 13.6230000),
    (100.5190000, 13.6290000),
    (100.5220000, 13.6330000),
    (100.5200000, 13.6440000),
    (100.5220000, 13.6540000),
    (100.5210000, 13.6610000),
    (100.5220000, 13.6650000),
    (100.5250000, 13.6680000),
    (100.5250000, 13.6710000),
    (100.5300000, 13.6670000),
    (100.5420000, 13.6650000),
    (100.5490000, 13.6680000),
    (100.5520000, 13.6710000),
    (100.5570000, 13.6860000),
    (100.5550000, 13.6990000),
    (100.5560000, 13.7020000),
    (100.5620000, 13.7020000),
    (100.5780000, 13.6960000),
    (100.5850000, 13.6890000),
    (100.5850000, 13.6840000),
    (100.5820000, 13.6780000),
    (100.5750000, 13.6700000),
    (100.5760000, 13.6660000),
    (100.5890000, 13.6610000),
    (100.5900000, 13.6590000),
    (100.6000000, 13.6540000),
    (100.6050000, 13.6540000),
    (100.6060000, 13.6520000),
    (100.6120000, 13.6500000),
    (100.6200000, 13.6510000),
    (100.6260000, 13.6480000),
    (100.6270000, 13.6460000),
    (100.6350000, 13.6430000),
    (100.6380000, 13.6440000),
    (100.6440000, 13.6500000),
    (100.6520000, 13.6480000),
    (100.6540000, 13.6500000),
    (100.6540000, 13.6580000),
    (100.6570000, 13.6670000),
    (100.6620000, 13.6670000),
    (100.6650000, 13.6630000),
    (100.6870000, 13.6540000),
    (100.6970000, 13.6520000),
    (100.7010000, 13.6530000),
    (100.7030000, 13.6620000),
    (100.7080000, 13.6700000),
    (100.7110000, 13.6840000),
    (100.7140000, 13.6900000),
    (100.7140000, 13.6960000),
    (100.7160000, 13.6990000),
    (100.7150000, 13.7130000),
    (100.7190000, 13.7140000),
    (100.7500000, 13.7120000),
    (100.7640000, 13.7130000),
    (100.7660000, 13.7110000),
    (100.7880000, 13.7130000),
    (100.8600000, 13.6850000),
    (100.8630000, 13.6870000),
    (100.8650000, 13.6960000),
    (100.8640000, 13.7000000),
    (100.8610000, 13.7010000),
    (100.8940000, 13.7540000),
    (100.9130000, 13.7880000),
    (100.9310000, 13.7980000),
    (100.9430000, 13.8150000),
    (100.9130000, 13.8390000),
    (100.9170000, 13.8480000),
    (100.9140000, 13.8510000),
    (100.9100000, 13.8510000),
    (100.9120000, 13.8960000),
    (100.9180000, 13.9460000),
    (100.9160000, 13.9500000),
    (100.7920000, 13.9360000),
    (100.7520000, 13.9220000),
    (100.6910000, 13.9200000),
    (100.6940000, 13.9320000),
    (100.6910000, 13.9350000),
    (100.6440000, 13.9350000),
    (100.6440000, 13.9390000),
    (100.6420000, 13.9410000),
    (100.6290000, 13.9420000),
    (100.6290000, 13.9500000),
    (100.6250000, 13.9540000),
    (100.6150000, 13.9590000),
    (100.6060000, 13.9590000),
    (100.6020000, 13.9570000),
    (100.6000000, 13.9540000),
    (100.6010000, 13.9510000),
    (100.5880000, 13.9560000),
    (100.5740000, 13.9590000),
    (100.5640000, 13.9540000),
    (100.5400000, 13.8520000),
    (100.5340000, 13.8500000),
    (100.5160000, 13.8330000),
    (100.5050000, 13.8270000),
    (100.5020000, 13.8180000),
    (100.5040000, 13.8130000),
    (100.5070000, 13.8120000),
    (100.4990000, 13.8040000),
    (100.4740000, 13.7950000),
    (100.4730000, 13.7980000),
    (100.4630000, 13.8020000),
    (100.4390000, 13.8040000),
    (100.3260000, 13.8080000)
]
line = geometry.LineString(polygoncoords)
polygon = geometry.Polygon(line)
xmin, ymin, xmax, ymax = polygon.bounds
n = 1000
x = np.arange(np.floor(xmin * n) / n, np.ceil(xmax * n) / n, 1 / n)
y = np.arange(np.floor(ymin * n) / n, np.ceil(ymax * n) / n, 1 / n)
points = geometry.MultiPoint(np.transpose(
    [np.tile(x, len(y)), np.repeat(y, len(x))]))
result = points.intersection(polygon)
individual_points = [(str(round(point.y, 3)) + '-' +
                      str(round(point.x, 3))) for point in result]
individual_points


# **Check BKK Function**

In [ ]:
def isBKK(month,day,hour):
    df = pd.read_csv("./2020"+month+"/2020"+month+day+"/2020"+month+day+"-"+hour+".csv")
    lat_lon = df["lat"].round(3).astype(str) + '_' + df["lon"].round(3).astype(str)
    df['lat-lon'] = df["lat"].round(4).astype(str) + '_' + df["lon"].round(4).astype(str)
    df['BKK']= lat_lon.isin(individual_points)
    # df.to_csv("./2020"+month+"/2020"+month+day+"/2020"+month+day+"-"+hour+".csv")

# **Import Drive**

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
cluster = pd.read_pickle('/content/gdrive/MyDrive/taxi-data/cluster.pickle')

# Data Manipulation
- Check BKK
- Add [ lat-lon ]
- Check ClusterID from cluster.pickle & Add [ ClusterID ] 
- Add [ pickup ] & [ dropoff ]

In [ ]:
def allInOne(month,day,hour):
  df = pd.read_csv('gdrive/My Drive/taxi-data/2020/2020'+month+'/2020'+month+day+'/2020'+month+day+'-'+hour+'.csv')
  df = df.drop(columns=['date','timeR'])  
  lat_lon = df["lat"].round(3).astype(str) + '-' + df["lon"].round(3).astype(str)
  df['lat-lon'] = df["lat"].round(4).astype(str) + '-' + df["lon"].round(4).astype(str)
  df['BKK']= lat_lon.isin(individual_points)
  df['clusterID'] = df['lat-lon'].apply(lambda x: cluster[x] if x in cluster.keys() else np.nan)
  df = df.sort_values(["VehicleID","timestamp"])
  pu = df[(df['VehicleID'] == df['VehicleID'].shift(-1)) & (df['for_hire_light'].shift(-1) - df['for_hire_light'] == -1)]
  do = df[(df['VehicleID'] == df['VehicleID'].shift(-1)) & (df['for_hire_light'].shift(-1) - df['for_hire_light'] == 1)]
  df['pickup'] = df.index.isin(pu.index)
  df['dropoff'] = df.index.isin(do.index)
  df.to_csv('gdrive/MyDrive/taxi-data/2020/202001V2/2020'+month+day+'/2020'+month+day+'-'+hour+'.csv')

In [ ]:
# from datetime import date, timedelta

# def daterange(start_date, end_date):
#     for n in range(int((end_date - start_date).days)):
#         yield start_date + timedelta(n)

# start_date = date(2020, 1, 1)
# end_date = date(2021, 1, 1)
# yearmonthdays = []
# months = []
# hours = []
# for single_date in daterange(start_date, end_date):
#     yearmonthdays.append(single_date.strftime("%Y%m%d"))
# for i in range(12):
#     months.append(str(i+1).zfill(2))
# for i in range(24):
#     hours.append(str(i).zfill(2))
# yearmonthdays.remove('20200331')
  

In [ ]:
import os

def mkdir(month,day,hour):
    path = '/content/gdrive/MyDrive/taxi-data/2021/2021'+month+'/2021'+month+day
    try:
        os.mkdir(path)
    except OSError as error:
        print(error)   

months = ['01','02','03','04','05','06','07','08','09','10','11','12']
hours = ['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
days_28 = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28']
days_30 = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30']
days_31 = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']
days_DEC = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','25','26','27','28','29','30','31']

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
def splitHour(month,day,hour):
  df = pd.read_csv('gdrive/My Drive/taxi-data/2021/2021'+month+'/2021'+month+day+'.csv', sep=',', 
                 names=['VehicleID', 'gpsvalid', 'lat', 'lon', 'timestamp', 'speed', 'heading', 'for_hire_light', 'engine_acc'])
  df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%dT%H:%M:%S') # convert to datetime
  df = df[(df['timestamp'].dt.month == int(month)) & (df['timestamp'].dt.day == int(day))] # check month & day
  df2 = df[(df['timestamp'].dt.hour == int(hour))] # check hour
  df2.to_csv('/content/gdrive/MyDrive/taxi-data/2021/2021'+month+'/2021'+month+day+'/2021'+month+day+'-'+hour+'.csv',index=False)

In [ ]:
for month in months:
    if month == '02':
        for day in days_28:
            for hour in hours:    
                splitHour(month,day,hour)
            print(day+'-'+month)
    elif month in ('04','06','09','11'):
        for day in days_30:
            for hour in hours:    
                splitHour(month,day,hour)
            print(day+'-'+month)
    elif month in ('01','03','05','07','08','10'):
        for day in days_31:
            for hour in hours:    
                splitHour(month,day,hour)
            print(day+'-'+month)
    elif month == '12':
        for day in days_DEC:
            for hour in hours:    
                splitHour(month,day,hour)
            print(day+'-'+month)            

In [ ]:
# for month in ['12']:
#     if month == '02':
#         for day in days_29:
#             for hour in hours:    
#                 allInOne(month,day,hour)
#             print(day+'-'+month)
#     elif month in ('04','06','09','11'):
#         for day in days_30:
#             for hour in hours:    
#                 allInOne(month,day,hour)
#             print(day+'-'+month)
#     elif month in ('01','03','05','07','08','10','12'):
#         for day in days_31:
#             for hour in hours:    
#                 allInOne(month,day,hour)
#             print(day+'-'+month)

# **Distance**

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive' , force_remount=True)

In [ ]:
import glob
path = r'/content/gdrive/MyDrive/2020/202003/202003**/*.csv'
#for fname in glob.glob(path):
#  df = pd.read_csv(fname)
#  df['VehicleID1'] = df['VehicleID'].shift(-1)
#  df.to_csv(fname)
#  print(fname)

for fname in glob.glob(path):
  df = pd.read_csv(fname)
  df['lat1'] = df['lat'].shift(-1)
  df['lon1'] = df['lon'].shift(-1)
  df['distance'] = df.apply(lambda x: haversine(x.lon, x.lat, x.lon1, x.lat1), axis=1)
  df.to_csv(fname)
  print(fname)

In [ ]:
import glob 
#import io
path = r'/content/gdrive/MyDrive/2020/202012/202012**/*.csv'
#path = r'/content/gdrive/MyDrive/2020/202009/20200920/20200920-04.csv'
#path = r'/content/test da.csv'
# from google.colab import files 
# uploaded = files.upload()
dis = 0
check = False
for fname in glob.glob(path):
  df = pd.read_csv(fname)
  for i , row in df.iterrows():
    if(row['VehicleID'] == row['VehicleID1']):
      if(row['pickup'] == True):
        check = True
      if(row['dropoff'] == True):
        check = False
        df.at[i,'distance'] = dis 
        dis = 0
      if(check == True):
        dis = dis + row['distance']
    else:
      check = False
      dis = 0
  df = df.loc[:, ~df.columns.str.contains('Unnamed')]
  df.to_csv(fname)
  print(fname)
#df.to_csv('test2.csv')
#from google.colab import files
#files.download("test2.csv")

In [ ]:
# import glob 
# path = r'/content/gdrive/MyDrive/2020/202001/202001**/*.csv'
# for fname in glob.glob(path):
#     df = pd.read_csv(fname)
#     df['timestamp'] = pd.to_datetime(df['timestamp'])
#     df['dayofweek'] = df['timestamp'].dt.dayofweek
    
#     print(df[['dayofweek'  , 'timestamp']])

# **DBSCAN**

* 01 : 5-11
* 02 : 16-22
* 03 : 15-21
* 04 : 19-25
* 05 : 17-23
* 06 : 7-13
* 07 : 12-18
* 08 : 16-22
* 09 : 6-12
* 10 : 4-10
* 11 : 8-14
* 12 : 13-19

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from pathlib import Path
import tqdm
import glob
import os
from datetime import datetime
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
# Mount Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/2020/clusters/cluster_pickup_details.csv')

In [ ]:
# df2 = df.groupby(['clusterID']).agg({"lat":"mean","lon":"mean","counts":"sum"})

df2 = df.groupby(['lat','lon']).agg({"counts":"sum"})

In [ ]:
df3 = df2.sort_values('counts', ascending=False).nlargest(300, 'counts')
df3

,,counts
lat,lon,
13.65634,100.60723,269
13.79531,100.69212,244
13.70383,100.53491,73
13.79723,100.53969,69
13.79045,100.75371,27
...,...,...
13.91901,100.60141,8
13.91864,100.60121,8
13.91939,100.60161,8


In [ ]:
df3 = df3.reset_index()

In [ ]:
import folium

# Define the center of the map
center_lat, center_lon = '13.771005', '100.621933'

# Create the map
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add markers to the map
for i, row in df3.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=row['counts'] / 100,
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(m)

# Show the map
m

In [ ]:
#select if pickup or dropoff
df = df[(df['pickup'] == True)]
df

In [ ]:
df = df.sort_values(["VehicleID","timestamp"])

In [ ]:
df['hour'] = df['timestamp'].str[11:13]
df['day'] = df['timestamp'].str[8:10] 
df

In [ ]:
df2 = df[["lat","lon"]]
df2

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
dbscan = DBSCAN(eps=0.001, min_samples=16)

In [ ]:
cluster = dbscan.fit(df2)
cluster

In [ ]:
df3 =  pd.DataFrame(cluster,columns=["clusterID"])

In [ ]:
df3

In [ ]:
df5 = pd.DataFrame.join(df2,df3)
df5 = df5.assign(counts=1)
df5 = df5.join(df["day"])
df5 = df5.join(df["hour"])
df5
df5 = df5[df5.clusterID != -1]

In [ ]:
df5

In [ ]:
df5.to_csv('/content/gdrive/MyDrive/2020/clusters/cluster_dropoff_details__0.001.csv',index=False)

In [ ]:
df6 = df5.groupby(['clusterID']).agg({"lat":"mean","lon":"mean","counts":"sum"})
df6

In [ ]:
df6.to_csv("/content/gdrive/MyDrive/2020/clusters/cluster_dropoff.csv",index=False)

# Dayofweek

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive' , force_remount=True)

Mounted at /content/gdrive


In [ ]:
import glob 
path = r'/content/gdrive/MyDrive/2020/202012/202012**/*.csv'
#path = r'/content/gdrive/MyDrive/2020/202001/20200104/20200104-05.csv'

for fname in glob.glob(path):
    df = pd.read_csv(fname)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['day'] = df['timestamp'].dt.dayofweek
    df['hour'] = df['timestamp'].dt.hour
    df.to_csv(fname,index = False)
    print(fname)

    #print(df[['timestamp','dayofweek' , 'day' , 'hour']])

In [ ]:
import glob 
from datetime import datetime
path = r'/content/gdrive/MyDrive/2020/202002/202002**/*.csv'
#path = r'/content/gdrive/MyDrive/2020/202009/20200920/20200920-04.csv' 
timeFirst = 0
timeLast = 0
rowi = 0

for fname in glob.glob(path):
  df = pd.read_csv(fname) #readfile
  df = df.drop(columns=['lat1','lon1']) 
  for i , row in df.iterrows(): #loop
    if(row['VehicleID'] == row['VehicleID1']):
      if(row['pickup'] == True): #ตัวรับ
        if(row['clusterID_pu']): #อยู่ในไอดีรับ
          timeFirst = datetime.strptime(row['timestamp'],"%Y-%m-%d %H:%M:%S") #เก็บค่าเวลา
          rowi = i # เก็บindex
      if(row['dropoff'] == True): #ตัวส่ง
        timeLast = datetime.strptime(row['timestamp'],"%Y-%m-%d %H:%M:%S") #เก็บค่าเวลา
        if(timeFirst != 0 and timeLast != 0):
          timediff_minute = timeLast.minute - timeFirst.minute
          timediff_second = abs(timeLast.second - timeFirst.second)/60
          timediff = timediff_minute + timediff_second #คิดค่าส่วนต่าง
          df.at[rowi,'traveltime'] = timediff #เขียนคอลัมน์ใหม่
          timeFirst = 0
          timeLast = 0
    else:
      timeFirst = 0
      timeLast = 0
  df.to_csv(fname,index = False)
  print(fname)
# df.to_csv('testtimetravel.csv')
# from google.colab import files
# files.download("testtimetravel.csv")

/content/gdrive/MyDrive/2020/202002/20200208/20200208-05.csv
/content/gdrive/MyDrive/2020/202002/20200208/20200208-18.csv
/content/gdrive/MyDrive/2020/202002/20200208/20200208-02.csv
/content/gdrive/MyDrive/2020/202002/20200208/20200208-11.csv
/content/gdrive/MyDrive/2020/202002/20200208/20200208-23.csv
/content/gdrive/MyDrive/2020/202002/20200208/20200208-13.csv
/content/gdrive/MyDrive/2020/202002/20200208/20200208-00.csv
/content/gdrive/MyDrive/2020/202002/20200208/20200208-03.csv
/content/gdrive/MyDrive/2020/202002/20200208/20200208-01.csv
/content/gdrive/MyDrive/2020/202002/20200208/20200208-10.csv
/content/gdrive/MyDrive/2020/202002/20200208/20200208-09.csv


In [ ]:
import glob 
from datetime import datetime

#path = r'/content/gdrive/MyDrive/2020/202012/202012**/*.csv'
path = r'/content/gdrive/MyDrive/2020/202009/20200920/20200920-04.csv'

mean = 0
for fname in glob.glob(path):
  df = pd.read_csv(fname)
  mean = df.groupby('clusterID_pu')['day'].mean()
print(mean)

# df.to_csv(fname,index = False)
#print(fname)
# print(df)
# df.to_csv('testtimetravel.csv')
# from google.colab import files
# files.download("testtimetravel.csv")

clusterID_pu
2.0      6.0
4.0      6.0
5.0      6.0
6.0      6.0
8.0      6.0
        ... 
292.0    6.0
294.0    6.0
295.0    6.0
296.0    6.0
297.0    6.0
Name: day, Length: 200, dtype: float64


# **Cleaning**

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
from pathlib import Path
import tqdm
import glob
import os
from datetime import datetime
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Mount Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = r'/content/gdrive/MyDrive/2020/202006/202006**/*.csv'

for fname in glob.glob(path):
  df = pd.read_csv(fname)
  df = df.loc[:, ~df.columns.str.contains('Unnamed')]
  df.to_csv(fname)
  print(fname)

In [ ]:
# # Directory path
# path = '/content/gdrive/MyDrive/2020/clusters'

# directories = []
# for d in os.listdir(dir_path):
#   directories.append(d)

# # List to store the dataframes
# df_list = []
# file_path = dir_path+ '/'

# # Loop through the directories in the main directory
# for dir in directories[7:14]:
#   for file in os.listdir(file_path+dir):
#     if file.endswith('.csv'):
#       df = pd.read_csv(os.path.join(file_path+dir, file))
#       df = df[(df['BKK'] == True)]
#       df = df[(df['pickup'] == True) | (df['dropoff'] == True)]
#       df = df.loc[:, ~df.columns.str.contains('Unnamed')]
#       df_list.append(df)

# df2 = pd.concat(df_list)

# df2.to_csv('/content/gdrive/MyDrive/2020/FEB.csv')

In [ ]:
import glob 
from datetime import datetime
timeFirst = 0
timeLast = 0
rowi = 0
subdirs = [ 
    '202009','202010', '202011', '202012'
]
for subdir in subdirs:
  path = r'/content/gdrive/MyDrive/2020/'+subdir+'/*/*.csv'
  for fname in glob.glob(path):
    df = pd.read_csv(fname) #readfile
    df = df.drop(columns=['lat1','lon1'],errors='ignore') 
    for i , row in df.iterrows(): #loop
      if(row['VehicleID'] == row['VehicleID1']):
        if(row['pickup'] == True): #ตัวรับ
          if(row['clusterID_pu']): #อยู่ในไอดีรับ
            timeFirst = datetime.strptime(row['timestamp'],"%Y-%m-%d %H:%M:%S") #เก็บค่าเวลา
            rowi = i # เก็บindex
        if(row['dropoff'] == True): #ตัวส่ง
          timeLast = datetime.strptime(row['timestamp'],"%Y-%m-%d %H:%M:%S") #เก็บค่าเวลา
          if(timeFirst != 0 and timeLast != 0):
            timediff_minute = timeLast.minute - timeFirst.minute
            timediff_second = abs(timeLast.second - timeFirst.second)/60
            timediff = timediff_minute + timediff_second #คิดค่าส่วนต่าง
            df.at[rowi,'traveltime'] = timediff #เขียนคอลัมน์ใหม่
            timeFirst = 0
            timeLast = 0
      else:
        timeFirst = 0
        timeLast = 0
    df.to_csv(fname,index = False)
    print(fname)
# df.to_csv('testtimetravel.csv')
# from google.colab import files
# files.download("testtimetravel.csv")

/content/gdrive/MyDrive/2020/202009/20200901/20200901-04.csv
/content/gdrive/MyDrive/2020/202009/20200901/20200901-05.csv
/content/gdrive/MyDrive/2020/202009/20200901/20200901-02.csv
/content/gdrive/MyDrive/2020/202009/20200901/20200901-03.csv
/content/gdrive/MyDrive/2020/202009/20200901/20200901-00.csv
/content/gdrive/MyDrive/2020/202009/20200901/20200901-01.csv
/content/gdrive/MyDrive/2020/202009/20200901/20200901-11.csv
/content/gdrive/MyDrive/2020/202009/20200901/20200901-10.csv
/content/gdrive/MyDrive/2020/202009/20200901/20200901-08.csv
/content/gdrive/MyDrive/2020/202009/20200901/20200901-09.csv
/content/gdrive/MyDrive/2020/202009/20200901/20200901-07.csv
/content/gdrive/MyDrive/2020/202009/20200901/20200901-06.csv
/content/gdrive/MyDrive/2020/202009/20200901/20200901-14.csv
/content/gdrive/MyDrive/2020/202009/20200901/20200901-12.csv
/content/gdrive/MyDrive/2020/202009/20200901/20200901-13.csv
/content/gdrive/MyDrive/2020/202009/20200901/20200901-21.csv
/content/gdrive/MyDrive/

# **Cluster**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
from datetime import datetime
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/2020/clusters/cluster_dropoff_details.csv')

In [ ]:
# df2 = df.groupby(['clusterID']).agg({"lat":"mean","lon":"mean","counts":"sum"})

df2 = df.groupby(['lat','lon']).agg({"counts":"sum"})

In [ ]:
df3 = df2.sort_values('counts', ascending=False).nlargest(300, 'counts')
df3 = df3.reset_index()
df3.insert(0, 'clusterID', range(len(df3)))
df3

In [ ]:
def get_sq(lat,lon,diff=0.0001,num=2):
  res_list = []
  for i in range(-num,num+1):
    for j in range(-num,num+1):
      res_list.append((lat+diff*i, lon+diff*j))
  
  return res_list

In [ ]:
INT_SPOTS = {}

for index, row in df[df['clusterID'].isin(df3.index)].iterrows():
  clusterID = row['clusterID']
  lat = row['lat']
  lon = row['lon']

  sq_latlons = get_sq(lat,lon,num=2)
  
  # cluster.loc[index, 'sqs'] = str(sq_latlons)

  for xlat, xlon in sq_latlons:
    xkey = str(round(xlat,4)) + '-' + str(round(xlon,4))
    INT_SPOTS[xkey] = int(clusterID)

In [ ]:
import pickle

with open('/content/gdrive/MyDrive/2020/clusters/DOclusterID.pickle', 'wb') as handle:
    pickle.dump(INT_SPOTS, handle, protocol=pickle.DEFAULT_PROTOCOL)

with open('/content/gdrive/MyDrive/2020/clusters/DOclusterID.pickle', 'rb') as handle:
    b = pickle.load(handle)

b

cluster labeling

In [ ]:
PUcluster = pd.read_pickle('/content/gdrive/MyDrive/2020/clusters/PUclusterID.pickle')
DOcluster = pd.read_pickle('/content/gdrive/MyDrive/2020/clusters/DOclusterID.pickle')

In [ ]:
subdirs = [
    '202003', '202004', '202005', '202006', '202007', '202008', '202009',
    '202010', '202011', '202012'
]
for subdir in subdirs:
  path = r'/content/gdrive/MyDrive/2020/'+subdir+'/*/*.csv'
  for fname in glob.glob(path):
    print(fname)
    df = pd.read_csv(fname)
    df = df.drop(columns=['gpsvalid','speed','heading','for_hire_light','engine_acc','clusterID'],errors='ignore', axis=1)
    df = df.loc[:, ~df.columns.str.contains('Unnamed')]
    df['clusterID_pu'] = df['lat-lon'].apply(lambda x: PUcluster[x] if x in PUcluster.keys() else np.nan)
    df['clusterID_do'] = df['lat-lon'].apply(lambda x: DOcluster[x] if x in DOcluster.keys() else np.nan)
    df.to_csv(fname,index=False)

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/2020/202010/20201005/20201005-00.csv')
d

,Unnamed: 0,VehicleID,gpsvalid,lat,lon,timestamp,speed,heading,for_hire_light,engine_acc,lat-lon,BKK,clusterID,pickup,dropoff,lat1,lon1,distance,VehicleID1
0,0,++iY830QzVXT6Slkr+8ftkTzMN4,1,13.77082,100.47394,2020-10-05 00:01:35,0,330,0,0,13.7708-100.4739,True,NaN,False,False,13.77082,100.47394,0.0,++iY830QzVXT6Slkr+8ftkTzMN4
1,1,++iY830QzVXT6Slkr+8ftkTzMN4,1,13.77082,100.47394,2020-10-05 00:04:35,0,330,0,0,13.7708-100.4739,True,NaN,False,False,13.77082,100.47394,0.0,++iY830QzVXT6Slkr+8ftkTzMN4
2,2,++iY830QzVXT6Slkr+8ftkTzMN4,1,13.77082,100.47394,2020-10-05 00:07:35,0,330,0,0,13.7708-100.4739,True,NaN,False,False,13.77082,100.47394,0.0,++iY830QzVXT6Slkr+8ftkTzMN4
3,3,++iY830QzVXT6Slkr+8ftkTzMN4,1,13.77082,100.47394,2020-10-05 00:10:35,0,330,0,0,13.7708-100.4739,True,NaN,False,False,13.77082,100.47394,0.0,++iY830QzVXT6Slkr+8ftkTzMN4
4,4,++iY830QzVXT6Slkr+8ftkTzMN4,1,13.77082,100.47394,2020-10-05 00:13:35,0,330,0,0,13.7708-100.4739,True,NaN,False,False,13.77082,100.47394,0.0,++iY830QzVXT6Slkr+8ftkTzMN4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123455,123455,zzLYPcDONaA8lLF2aJYFKnoRDQ4,1,13.82464,100.52877,2020-10-05 00:47:16,0,325,1,0,13.8246-100.5288,True,NaN,False,False,13.82464,100.52877,0.0,zzLYPcDONaA8lLF2aJYFKnoRDQ4
123456,123456,zzLYPcDONaA8lLF2aJYFKnoRDQ4,1,13.82464,100.52877,2020-10-05 00:50:16,0,325,1,0,13.8246-100.5288,True,NaN,False,False,13.82464,100.52877,0.0,zzLYPcDONaA8lLF2aJYFKnoRDQ4
123457,123457,zzLYPcDONaA8lLF2aJYFKnoRDQ4,1,13.82464,100.52877,2020-10-05 00:53:16,0,325,1,0,13.8246-100.5288,True,NaN,False,False,13.82464,100.52877,0.0,zzLYPcDONaA8lLF2aJYFKnoRDQ4
123458,123458,zzLYPcDONaA8lLF2aJYFKnoRDQ4,1,13.82464,100.52877,2020-10-05 00:56:16,0,325,1,0,13.8246-100.5288,True,NaN,False,False,13.82464,100.52877,0.0,zzLYPcDONaA8lLF2aJYFKnoRDQ4


In [ ]:
d = d.drop(columns=['gpsvalid','speed','heading','for_hire_light','engine_acc','clusterID'],errors='ignore', axis=1)

In [ ]:
d['clusterID_pu'] = d['lat-lon'].apply(lambda x: PUcluster[x] if x in PUcluster.keys() else np.nan)
d['clusterID_do'] = d['lat-lon'].apply(lambda x: DOcluster[x] if x in DOcluster.keys() else np.nan)

In [ ]:
d
